## Pendahuluan

**Studi: Prediksi Paket Seluler Menggunakan Machine Learning**

Sebuah operator seluler bernama `Megaline` ingin mengatasi kepuasan pelanggan yang masih menggunakan paket lama dengan mengembangkan model `machine learning`. Model ini akan menganalisis perilaku konsumen dan merekomendasikan paket terbaru, `Smart` atau `Ultra`. `Megaline` memiliki dataset berisi perilaku pengguna paket `Smart` dan `Ultra`. 

Proyek ini memiliki beberapa tujuan dan rumusan masalah:

- Menentukan algoritma terbaik untuk model `machine learning` pada dataset `Megaline`.
- Menemukan hyperparameter terbaik untuk model `machine learning`.
- Memeriksa apakah model `machine learning` yang dipilih dapat memenuhi uji kelayakan (sanity check).
- Mengevaluasi apakah model `machine learning` yang dipilih dapat digunakan untuk menguji sampel data sembarang.

## Konten <a id='back'></a>

* [1. Pendahuluan](#intro)
* [2. Konten](#back)
  * [2.1 Mempersiapkan Dataset](#cont_1)
    * [2.1.1 Memuat Library](#cont_2) 
    * [2.1.2 Memuat Dataset](#cont_3)
    * [2.1.2 Mengecek Nilai Hilang](#cont_3)
    * [2.1.3 Mengecek Duplikasi](#cont_4) 
    * [2.1.4 Mengubah Tipe Data](#cont_5) 
  * [2.2 Membuat Model Machine Learning](#cont_6)
    * [2.2.1 Pembagian Dataset](#cont_7) 
    * [2.2.2 Melatih dan Menguji Algoritma Machine Learning](#cont_8) 
        * [2.2.2.1 Algoritma Klasifikasi Decision Tree](#cont_9) 
        * [2.2.2.2 Algoritma Klasifikasi Random Forest](#cont_10) 
        * [2.2.2.3 Algoritma Regresi Logistik](#cont_11) 
    * [2.2.3 Model Dengan Algoritma Terbaik](#cont_12) 
    * [2.2.4 Menguji Kelayakan Model (Sanity Check)](#cont_13) 
* [3. Aplikasi Model Machine Learning](#cont_14)
* [4. Kesimpulan Umum](#cont_15)

### Mempersiapkan Dataset

Tahap pertama yang perlu dilakukan adalah mempersiapkan dataset mulai dari memuat library yang diperlukan, memuat dataset , mengecek sampel data, mengecek nilai yang hilang, mengecek duplikat dan mengecek tipe data.

#### Memuat Library

Kita embutuhkan dua library yaitu `pandas` untuk mengolah dataset dan `scikit learn` untuk pemodelan `machine learning`. Mari kita muat library keduanya.

In [1]:
# memuat library
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

#### Memuat Dataset

Mari kita lanjutkan memuat dataset yang dimiliki oleh `Megaline`.

In [2]:
# memuat dataset megaline
df_megaline = pd.read_csv('/datasets/users_behavior.csv')

Selanjutnya kita akan menampilkan informasi dan sampel data dari dataset `Megaline`.

In [3]:
# mengecek informasi data
print(df_megaline.info())

# mengecek sampel data
df_megaline.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


,calls,minutes,messages,mb_used,is_ultra
1561,59.0,486.67,0.0,13432.30,1
360,81.0,566.10,0.0,23426.08,0
948,58.0,386.44,0.0,16581.86,0
2057,37.0,291.42,0.0,16362.24,0
2149,41.0,239.53,53.0,25004.15,1


Dataset berisi kolom-kolom berikut:
- `calls` merupakan jumlah panggilan
- `minutes` merupakan durasi panggilan dalam satuan menit
- `messages` merupakan jumlah pesan
- `mb_used` merupakan jumlah penggunaan data internet dalam satuan MB
- `is_ultra` merupakan kolom penentu apakah pengguna menggunakan paket ultra (bernilai 1) atau tidak (bernilai 0)

#### Mengecek Nilai Hilang

Mari kita pastikan apakah ada data yang hilang pada dataset.

In [4]:
# Cek apakah ada data nilai yang hilang

df_megaline_missed = df_megaline.isna().sum().to_frame()
df_megaline_missed = df_megaline_missed.rename(columns={0: 'missing_values'})
df_megaline_missed['% of total'] = (df_megaline_missed['missing_values'] / df_megaline.shape[0]).round(2)
df_megaline_missed.sort_values(by='missing_values', ascending=False)

,missing_values,% of total
calls,0,0.0
minutes,0,0.0
messages,0,0.0
mb_used,0,0.0
is_ultra,0,0.0


Tidak ada nilai yang hilang pada dataset.

#### Mengecek Duplikasi

Kita lanjutkan mengecek duplikasi pada dataset.

In [5]:
# mengecek duplikasi
df_megaline.duplicated().sum()

0

Dari hasil pengecekan tidak ditemukan adanya duplikasi pada dataset.

#### Mengubah Tipe Data 

Berdasarkan pengamatan sebelumnya, kita akan mengubah tipe data pada kolom `calls` dan `messages` dari tipe `float` menjadi `integer`.

In [6]:
# mengubah tipe data kolom calls menjadi integer
df_megaline['calls'] = df_megaline['calls'].astype('int')

# mengubah tipe data kolom messages menjadi integer
df_megaline['messages'] = df_megaline['messages'].astype('int')

# mengecek tipe data yang baru
df_megaline.info()

# menampilkan sampel data
df_megaline.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
765,45,320.44,117,35689.60,1
2249,46,314.16,48,16684.31,1
840,111,707.25,77,12279.31,0
2806,107,733.45,98,10756.78,1
1234,5,34.58,7,10305.72,0


Kolom `calls` dan `messages` sudah berhasil diubah menjadi tipe data `integer`.

### Membuat Model Machine Learnin|g

Kita lanjutkan membuat `model machine learning` yang terbaik diperlukan beberapa tahapan diantaranya: 

- Pembagian Dataset
- Memilih Algoritma 
- Menguji Algoritma
- `Tuning Hyperparamater`.

Beberapa `algoritma machine learning` akan kita uji dan memilih yang paling efektif digunakan. Beberapa algoritma tersebut yaitu:
- Algoritma Klasifikasi Decision Tree
- Algoritma Klasifikasi Random Forest
- Algoritma Regresi Logistik

Kita akan menentukan paket apa yang direkomendasikan sebagai `target` berdasarkan beberapa `fitur` yang berkaitan. Fitur-fitur tersebut diantranya `calls`, `minutes`, `messages` dan `mb_used`.

Karena kita hanya memilki satu buah dataset, kita akan membaginya kedalam beberapa kelompok, untuk membuat `model machine learning`. Pembagian dipecah menjadi `60%` dataset untuk `training`, `20%` dataset untuk `validasi` dan `20%` untuk `testing`.

#### Pembagian Dataset

In [7]:
# membagi dataset menjadi 60% training, 20% validasi dan 20% test

# membagi dataset megaline menjadi 60% untuk training dan 40% untuk (validasi + test)
df_train, df_temp = train_test_split(df_megaline, test_size=0.6, random_state=155)

# membagi dataset temporary menjadi 50% untuk validation dan 50% untuk test
df_validation, df_test = train_test_split(df_temp, test_size=0.5, random_state=155)

In [8]:
# membagi dataset training menjadi fitur dan target
features_train = df_train.drop(['is_ultra'],axis=1)
target_train = df_train['is_ultra']

# membagi dataset validation menjadi fitur dan target
features_valid = df_validation.drop(['is_ultra'],axis=1)
target_valid = df_validation['is_ultra']

# membagi dataset test menjadi fitur dan target
features_test = df_test.drop(['is_ultra'],axis=1)
target_test = df_test['is_ultra']

# menampilkan shape dari training dan validation
print('features_train:',features_train.shape)
print('target_train:',target_train.shape,'\n')

print('features_valid:',features_valid.shape)
print('target_valid:',target_valid.shape,'\n')

print('features_test:',features_test.shape)
print('target_test:',target_test.shape)

features_train: (1285, 4)
target_train: (1285,) 

features_valid: (964, 4)
target_valid: (964,) 

features_test: (965, 4)
target_test: (965,)


#### Melatih dan Menguji Algoritma Machine Learning

Dari fitur dan target pada pembagian dataset sebelumnya, kita simpulkan `model machine learning` yang bisa dibuat adalah tipe `supervised learning - klasifikasi `. Mari kita latih dan uji model-model yang sudah disebutkan sebelumnya:

##### Algoritma Klasifikasi Decision Tree

Selanjutnya kita menguji `algoritma klasifikasi decision tree` dimana `hyperparameter` untuk kedalaman pohon akan kita uji untuk mendapatkan `hyperparameter` yang terbaik.

In [9]:
# percobaan algoritma decision tree

# membuat temporary variabel
best_result = 0
best_depth = 0

# pengujian kedalaman model decision tree (depth -> 1 ~ 50)
for depth in range(1, 51):
    # membuat model decision tree
    model = DecisionTreeClassifier(random_state=155, max_depth=depth) 
    # melatih model menggunakan features dan target train
    model.fit(features_train, target_train)
    # menghitung akurasi menggunakan features dan target validation
    result = model.score(features_valid,target_valid)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth

# menampilkan output        
print("Akurasi Model Terbaik (Decision Tree)")
print(f"Akurasi\t: {best_result:.5f}")
print(f"Depth\t: {best_depth}")

Akurasi Model Terbaik (Decision Tree)
Akurasi	: 0.80187
Depth	: 7


Dari sini bisa kita kita lihat akurasi skor terbaik yang diperoleh sebesar `80,19%` pada kedalaman pohon `depth` sebanyak `7`.


##### Algoritma Klasifikasi Random Forest

Dengan cara yang sama kita akan menguji `algoritma klasifikasi decision tree` dimana `hyperparameter` untuk `random forest` yang akan kita `tuning` adalah jumlah pohon `n_estimators` dan kedalaman pohon `depth`. 

Disini kita akan coba untuk kedalaman pohon `1 hingga 10`, dan jumlah pohon `1 hingga 15` yang akan kita cari `hyperparameter` terbaiknya.

In [10]:
# percobaan algoritma random forest
# kedalaman pohon: depth -> 1 ~ 10
# jumlah pohon: n_estimators -> 1 ~ 15

# membuat temporary variabel
best_result = 0
best_est = 0
best_depth = 0

# pengujian kedalaman model dan jumlah pohon 
# pengaturan jumlah pohon
for est in range(1, 16):
    # pengaturan jumlah kedalaman pohon
    for depth in range (1, 11):
        # membuat model random forest classifier
        model = RandomForestClassifier(random_state=155, n_estimators=est, max_depth=depth)
        # melatih model menggunakan features dan target train
        model.fit(features_train, target_train)
        # menghitung akurasi menggunakan features dan target validation
        result = model.score(features_valid,target_valid)
        if result > best_result:
            best_result = result
            best_est = est
            best_depth = depth

# menampilkan output        
print("Akurasi Model Terbaik (Random Forest)")
print(f"Akurasi\t\t: {best_result:.5f}")
print(f"Depth\t\t: {best_depth}")
print(f"N_Estimators\t: {best_est}")

Akurasi Model Terbaik (Random Forest)
Akurasi		: 0.82158
Depth		: 9
N_Estimators	: 13


Bisa dilihat kita bisa mendapatkan akurasi skor sebesar `82,16%` hanya menggunakan kedalaman pohon `9` dan jumlah pohon `13`.

Mari kita coba tingkatkan `hyperparameter` apakah kita bisa mendapatkan yang lebih baik akurasi skornya dibandingkan skor ini.

Kita akan coba tingkatkan kedalaman pohon `1 hingga 20`, dan jumlah pohon `1 hingga 70 (inkremen 5)` yang akan kita cari `hyperparameter` terbaiknya.

In [11]:
# percobaan algoritma random forest
# kedalaman pohon: depth -> 1 ~ 20
# jumlah pohon: n_estimators -> 1 ~ 70 {increment 5}

# membuat temporary
best_result = 0
best_est = 0
best_depth = 0

# pengujian kedalaman model dan jumlah pohon
# pengaturan jumlah pohon
for est in range(1, 71, 5):
    # pengaturan jumlah kedalaman pohon
    for depth in range (1, 21):
        # membuat model random forest classifier
        model = RandomForestClassifier(random_state=155, n_estimators=est, max_depth=depth)
        # melatih model menggunakan features dan target train
        model.fit(features_train, target_train)
        # menghitung akurasi menggunakan features dan target validation
        result = model.score(features_valid,target_valid)
        if result > best_result:
            best_result = result
            best_est = est
            best_depth = depth

# menampilkan output        
print("Akurasi Model Terbaik (Random Forest)")
print(f"Akurasi\t\t: {best_result:.5f}")
print(f"Depth\t\t: {best_depth}")
print(f"N_Estimators\t: {best_est}")

Akurasi Model Terbaik (Random Forest)
Akurasi		: 0.82261
Depth		: 9
N_Estimators	: 21


Bisa dilihat kita bisa mendapatkan akurasi skor sebesar `82,26%` hanya menggunakan kedalaman pohon `9` dan jumlah pohon `21`.

Disini kita simpulkan semakin tinggi `hyperparameter` tidak menyebabkan kenaikan akurasi skor yang signifikan, bahkan hampir sama. Maka kita ambil `hyperparameter` terbaik untuk `algoritma random forest` kali ini pada kedalaman pohon `9` dan jumlah pohon `13`.

##### Algoritma Regresi Logistik

Dengan cara yang sama kita akan menguji `algoritma regresi logistik` menggunakan `solver` 'liblinear'.

In [12]:
# percobaan algoritma regresi logistik

# membuat model regresi logistik
model = LogisticRegression(random_state=155, solver='liblinear') 

# melatih model menggunakan features dan target train
model.fit(features_train, target_train)

# menghitung akurasi menggunakan features dan target validation
result = model.score(features_valid,target_valid)

# menampilkan output        
print("Akurasi Model Terbaik (Regresi Logistik)")
print(f"Akurasi\t: {result:.5f}")

Akurasi Model Terbaik (Regresi Logistik)
Akurasi	: 0.68154


Disini kita mendapatkan skor yang lebih rendah dari kedua algoritma sebelumnya yang kita ujikan yaitu hanya `68.15%`.

#### Model Dengan Algoritma Terbaik

Dari pengujian sebelumnya, kita memilih algoritma terbaik dengan `hyperparameter`nya adalah `Algoritma Klasifikasi Random Forest` dengan kedalaman pohon `depth` sebanyak `9` dan jumlah pohon sebanyak `13`.

Model tersebut hanya menggunakan `60%` dataset sebagai latihan, bagaimana jika kita tingkatkan menjadi `80%`, tentu berharap model akan bisa memprediksi dengan lebih baik.

Mari kita gabungkan dataset training dan validasi menjadi satu dataset.

In [13]:
# melatih model menggunakan dataset training dan validasi untuk mendapatkan hasil yang lebih akurat menggunakan random forest

# menggabungkan dataset training dan validasi
merge_df = pd.concat([df_train,df_validation],axis=0)
print('New Dataset:',merge_df.shape)

# membagi dataset akhir menjadi fitur dan target
features = merge_df.drop(['is_ultra'],axis=1)
target = merge_df['is_ultra']

# max_depth = 9
# n_estimators = 13
best_model = RandomForestClassifier(random_state=155, n_estimators=13, max_depth=9)

# melatih model terbaik
best_model.fit(features,target)

New Dataset: (2249, 5)


RandomForestClassifier(max_depth=9, n_estimators=13, random_state=155)

#### Menguji Kelayakan Model (Sanity Check)

Mari kita uji kelayakanya `sanity check` menggunakan `dataset testing` yang sudah dibuat sebelumnya dimana jawaban yang benar tidak dimasukan pada saat pelatihan model terbaik.

In [14]:
# uji kelayakan model (sanity check)
accuracy = best_model.score(features_test,target_test)

# menampilkan output
print("Akurasi best_model:",accuracy)

Akurasi best_model: 0.8103626943005181


Bisa dilihat ternyata akurasinya masih bertahan disekitar `81%` dan masih diatas ambang batas akurasi skor `75%`.

### Aplikasi Model Machine Learning

Disini kita akan membuat sembarang dataset dimana kita akan mengetahui paket mana yang cocok jika kita memiliki fitur-fitur seperti berikut.

In [15]:
# membuat sembarang dataframe untuk menguji model yang terpilih
data_test = pd.DataFrame({
    'calls':[70,20,50,100,90],
    'minutes':[100,85,300,250,30],
    'messages':[50,35,300,60,500],
    'mb_used':[10000,500,7000,3000,1000]
})

# menampilkan dataframe
data_test

,calls,minutes,messages,mb_used
0,70,100,50,10000
1,20,85,35,500
2,50,300,300,7000
3,100,250,60,3000
4,90,30,500,1000


Mari kita prediksi paketnya untuk kelima pengguna tersebut.

In [16]:
# memprediksi data_test
best_model.predict(data_test)

array([0, 0, 1, 1, 1])

Bisa dilihat kita bisa memprediksi user yang menggunakan paket `smart` sebanyak 2 user dan paket `ultra` sebanyak 3 user.

### Kesimpulan Umum

Dari projek ini kita sudah memuat library yang diperlukan, mempersiapkan dataset, membagi dataset, melatih serta menguji `model machine learning` untuk memprediksi paket yang direkomendasikan kepada user berdasarkan prilaku user.

Dan didapatkan kesimpulan sebagai berikut:
- Pembagian dataset untuk melatih serta menguji `model machine learning` dibagi menjadi `60%` untuk `training`, `20%` untuk `validasi` dan `20%` untuk `testing`.
- Algoritma yang diujikan diantaranya: `Klasifikasi Decision Tree`, `Klasifikasi Random Forest` dan `Regresi Logistik`.
- Algoritma terbaik dengan `hyperparameter`nya adalah `Algoritma Klasifikasi Random Forest` dengan kedalaman pohon `depth` sebanyak `9` dan jumlah pohon sebanyak `13` menghasilkan akurasi skor `82.16%`.
- Pengujian kelayakan `Sanity Check` dilakukan dan model mampu mempertahankan akurasi skornya sebesar `81.03%`.

<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Hebat, Kerja bagus!

</div>